**Function Call**

-GPT가 사용할 수 있는 함수/툴에 대해 설명하는 시스템 프롬프트

-함수를 정의하고

-함수가 언제 사용되는지, 함수를 실행하기 위해서 어떤 인자가 필요한지 JSON Schema 형식으로 제공한다.

**Function Call의 동작 순서**

1. 유저의 쿼리/메시지 입력된다.

2. GPT는 유저 메시지에 따라 특정 함수를 사용해야 한다고 판단하면,

실행할 함수의 이름

함수를 실행할 때 필요한 인풋(Argument)

를 제공한다.

3. LLM이 제공받은 함수 이름과 인자를 통해 함수를 실행한다.

4. 실행된 함수의 결과를 LLM에게 다시 알려주고 답변을 뽑아낸다.

핵심: LLM이 툴을 사용할 때는 꼭 2번의 호출이 일어나야 한다!!!

In [ ]:
!pip install streamlit
!pip install openai
!pip install pyngrok

In [ ]:
!ngrok authtoken 2di7Hwxpn6OAYj9NchuDCoKosgR_2QejNWzh68Tjn3KfHiWUo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py

import streamlit as st
from openai import OpenAI
import json
import requests

API_KEY = ""
weather_api_key= ""

st.title("날씨봇🌤️")

# Set OpenAI API key from Streamlit secrets
client = OpenAI(api_key=API_KEY)

# Set a default model
if "openai_model" not in st.session_state:
    st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [{
      "role": "system",
      "content": "특정 지역의 날씨 정보를 묻는 질문에 답변하는 챗봇입니다.\
      함수에 연결할 값을 가정하지 마세요.\
      사용자의 요청이 모호한 경우 설명을 요청하세요\
      한국어로 친절하게 이모지를 섞어서 답변해줘"
      }]


def get_current_weather(location, format):
    # Replace 'your_api_key_here' with your actual OpenWeatherMap API key
    api_key = weather_api_key
    # OpenWeatherMap API endpoint for current weather
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Convert the format to the corresponding unit system for the API request
    units = 'metric' if format == 'celsius' else 'imperial'

    # Construct the complete API URL with parameters
    complete_url = f"{base_url}appid={api_key}&q={location}&units={units}"

    # Perform the API request
    response = requests.get(complete_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()

        # Extract relevant weather information
        temperature = data['main']['temp']
        weather_description = data['weather'][0]['description']

        # Construct a result string
        result = (f"'location:{location}, 'temperature':{temperature},'format':{format}, 'weather_description':{weather_description}")
        return result
    else:
        # Handle possible errors (e.g., location not found)
        return "Weather information could not be retrieved. Please check the location and try again."

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather(e.g. temperature, humidity)",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Seoul(서울), Busan(부산).",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    }]


# Display chat messages from history on app rerun
for message in st.session_state.messages:
    if message["role"] != "system":
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# Accept user input for chat
if prompt := st.chat_input("날씨에 대해 물어보세요"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        response = client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=st.session_state.messages,
            tools=tools,
            )
        response_message = response.choices[0].message
        function_name = response_message.tool_calls[0].function.name
        arguments = json.loads(response_message.tool_calls[0].function.arguments)
# 함수에 인자값을 넣어 실행하기
        function_response = eval(f"{function_name}(**arguments)")

        st.session_state.messages.append({
          "role":"function",
          "name":function_name,
          "content":function_response
        })

        # 여기부터 시작해야 댈듯
        for response in client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=st.session_state.messages,
            tools = tools,
            stream=True,
        ):
            full_response += (response.choices[0].delta.content or "")
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})

Overwriting app.py


In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

<NgrokTunnel: "https://7ca5-34-125-106-56.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      6 ?        00:00:03 node
     16 ?        00:00:00 oom_monitor.sh
     18 ?        00:00:00 run.sh
     20 ?        00:00:00 kernel_manager_
     22 ?        00:00:00 tail
     60 ?        00:00:08 python3 <defunct>
     61 ?        00:00:00 colab-fileshim.
     83 ?        00:00:04 jupyter-noteboo
     84 ?        00:00:00 dap_multiplexer
    548 ?        00:00:04 python3
    579 ?        00:00:00 python3
   1124 ?        00:00:00 language_servic
   1129 ?        00:01:08 node
   1622 ?        00:00:03 streamlit
   1623 ?        00:00:01 ngrok
   2268 ?        00:00:00 streamlit
   2436 ?        00:00:00 sleep
   2437 ?        00:00:00 ps


In [ ]:
!kill 2268